## Hartree-Fock Class
<p>
    <br />
    Hartree Fock class that implements the Hartree-Fock method. Designed to allow for automation and improved testing of HF calculations.
</p>    

In [1]:
import numpy as np
from molecule import atom
from molecule import vector
from molecule import gaussian
from molecule import molecule
from notebookImporter import importNotebook

class HF():
    
    #import integrals notebook for the hartree method
    integrals = importNotebook("Hartree_Integrals")
    scf = importNotebook("Hartree_SCF")
    
    #declare all global variables here
    SCF_CONVERGENCE = 0
    MAX_CYCLE = 0
    molecularSystem = 0
    S = 0
    T = 0
    V = 0
    electronRepulsion = 0
    H = 0
    size = 0
    X = 0
    F = 0
    E = []
    totalE = 0
    MOEnergy = []
    converged = False
    verbose = False
    
    #constructor for the HF class
    #molecularSystem, molecule object with the system to perform HF on 
    #SCF_CONVERGENCE, user can optoinally specficy the differnce in energy required for SCF convergence
    #MAX_CYLCE, the maximum number of SCF cycles that can be taken before SCF convergence fails
    #verbose: Boolean to represent whether certain prints should be executed during the run of this class
    def __init__(self, molecularSystem, SCF_CONVERGENCE=pow(10, -50), MAX_CYCLE=100, verbose=False):
        
        self.SCF_CONVERGENCE = SCF_CONVERGENCE
        self.MAX_CYCLE = MAX_CYCLE
        self.molecularSystem = molecularSystem
        self.verbose = verbose
        
        #compute all the necessary integral for this computation
        #order of integral computations are as follows:
        #Overlap, Electron KE, Nuclear Attraction, Electron Repulsion
        self.S = self.integrals.overlap(molecularSystem)
        self.T = self.integrals.kineticEnergy(molecularSystem)
        self.V = self.integrals.nuclearAttraction(molecularSystem)
        self.electronRepulsion = self.integrals.electronElectronRepulsion(molecularSystem)
        
        #create the electronic Hamiltonian
        self.H = np.matrix(self.T)
        for atom in self.V:
            self.H += np.matrix(atom)
        
        #prepare for SCF computation
        self.size = len(self.S)
        self.X = self.scf.X(self.S, self.size)
        self.F = self.H
        
        self.E = []
        self.totalE = 0
        self.converged = False
        
#--------------------------------------------------------------------------------
    
    #method to begin the SCF procedure of the Hartree method
    #returns total energy of the system
    def SCF(self):

        while( not self.converged ):

            #convert Fock Matrix it to MO basis 
            self.F = self.X.transpose() * self.F * self.X 
        
            #diagnolize the Fock Matrix to obtain the MOs and the their respective energies
            self.MOEnergy, MO = np.linalg.eigh(self.F)

            #Transform the MO basis MOs to an AO basis
            C = self.X * MO
    
            #compute the electron density, the two electron term, and then use G to compute the new Fock matrix
            P = self.scf.densityMatrix(C, self.molecularSystem.N, self.size)
            G = self.scf.G(self.electronRepulsion, P, self.size)
            self.F = self.H + G
    
            #compute the new expectation energy 
            self.E.append(self.scf.expectationEnergy(self.H, self.F, P))
    
            #display information about current SCF iteration to the user
            sizeE = len(self.E)
            
            if(self.verbose):
                print("SCF Iteration #" + str(sizeE) + ", Energy Value: " + str(self.E[sizeE-1]) + " Hartrees")
            
            #check if at least two SCF iterations have occured
            #if more than two have occured, then check if the difference betweeen this E, 
            #and the previous E is less then the covergence value, if yes, end the SCF loop
            if(len(self.E) > 2):
                if(abs(self.E[sizeE-2] - self.E[sizeE-1]) < self.SCF_CONVERGENCE):
                    self.converged = True
                    
                #check if maximum of cycles have been surpassed
                elif(sizeE >= self.MAX_CYCLE):
                    print("SCF Did not Converge!!\n\n")
                    break;
                    
        #add in nuclear-nuclear repulsion to the energy
        self.totalE = self.E[sizeE-1] + self.scf.nuclearRepulsion(self.molecularSystem)
        
        if(self.verbose):
            print("Total E: " + str(self.totalE))
            print("MO Energies: " + str(self.MOEnergy))
            print("\n")
            
        return self.totalE